In [3]:
#Imports
import random
import arcade
import os
from pathlib import Path
from enum import Enum
from PIL import Image

In [4]:
#configuration
Sprite_part = Enum('Sprite_part',['BEHIND', 'BODY', 'FEET', 'LEGS', 'TORSO', 'BELT', 'HEAD', 'HANDS', 'SHEILD', 'WEAPON'])
Action = Enum('Action',['shoot','hurt','slash','slash_reverse','spellcast','thrust','walk']) #slash_reverse.  not supported

#what actions use the weapon art?  Is the art oversized?
# weapon_actions ={
#     'dagger': ,
#     'long_sword':,
#     'spear':,
#     'crystal_staff':,
#     'bow':,
#     'crossbow':
    
# }
sprite_part_options = {
    Sprite_part.BEHIND:['quiver'],
    Sprite_part.BODY:['human', 'skeleton','goblin','boar','lizard','minotaur','troll','wolf'],
    Sprite_part.FEET:['plate_armor_shoes', 'shoes_brown'],
    Sprite_part.LEGS:['pants_greenish', 'plate_armor_pants', 'robe_skirt'],
    Sprite_part.TORSO:['chain_armor_jacket_purple', 'chain_armor_torso', 'leather_armor_bracers', 'leather_armor_shirt_white', 'leather_armor_shoulders', 'leather_armor_torso', 'plate_armor_arms_shoulders', 'plate_armor_torso', 'robe_shirt_brown'],
    Sprite_part.BELT:['leather', 'rope'],
    Sprite_part.HEAD:['chain_armor_helmet', 'chain_armor_hood', 'hair_blonde', 'leather_armor_hat', 'plate_armor_helmet', 'robe_hood'],
    Sprite_part.HANDS:['plate_armor_gloves'],
    Sprite_part.WEAPON:['dagger', 'spear', 'crystal_staff', 'bow', 'arrow', 
                        'shield_cutout_body', 'shield_cutout_chain_armor_helmet']}

std_sprite_wd, std_sprite_ht = 64,64
sprite_sheet_dims = { #columns, rows for normal
    Action.shoot:(13,4),
    Action.hurt:(6,1),
    Action.slash:(6,4),
    Action.spellcast:(7,4),
    Action.slash_reverse:(6,4),
    Action.thrust:(8,4),
    Action.walk:(9,4)
}
ov_sprite_wd = 192

textures = {}

In [14]:
#Split function 
#breaks up large png into smaller action based pngs in proper dirs
def split_png (base_name, sprite_part, crop_order):
    root_path = Path('.')/ 'textures' / 'png'
    source_path = root_path / '01_raw' / f'{sprite_part}' / f'{base_name}.png'
    img = Image.open(source_path.as_posix())
    lower = 0
    for i,thruple in enumerate(crop_order):
        a,is_full,is_ov = thruple
        cols,rows = sprite_sheet_dims[a]
        left = 0
        if is_ov:
            sprite_wd, sprite_ht = 192,192
        else:
            sprite_wd, sprite_ht = 64,64
        upper = lower #i*4*sprite_ht
        right = cols*sprite_wd
        lower = upper + rows*sprite_ht
        box = (0,upper,right,lower)
        print(f'{a.name}:rows,cols:{(rows,cols)}, full:{bool(is_full)}, ov:{bool(is_ov)}, box:{(0,upper,right,lower)}')
        if is_full:
            t_img = img.crop(box)
            save_path = root_path / '02_source' / a.name /  f'{sprite_part}_{base_name}.png'
            t_img.save(save_path.as_posix())

In [16]:
#weapons = ['long_sword','crystal_staff','spear','crossbow']
long_sword_crop_order = [
    (Action.spellcast,0,0),
    (Action.thrust,0,0), 
    (Action.walk,1,0),
    (Action.slash,0,0), 
    (Action.shoot,0,0), 
    (Action.hurt,1,0),
    (Action.slash,1,1), 
    (Action.slash_reverse,1,1),
    (Action.thrust,1,1)]
split_png('long_sword','WEAPON',long_sword_crop_order)

spellcast:rows,cols:(4, 7), full:False, ov:False, box:(0, 0, 448, 256)
thrust:rows,cols:(4, 8), full:False, ov:False, box:(0, 256, 512, 512)
walk:rows,cols:(4, 9), full:True, ov:False, box:(0, 512, 576, 768)
slash:rows,cols:(4, 6), full:False, ov:False, box:(0, 768, 384, 1024)
shoot:rows,cols:(4, 13), full:False, ov:False, box:(0, 1024, 832, 1280)
hurt:rows,cols:(1, 6), full:True, ov:False, box:(0, 1280, 384, 1344)
slash:rows,cols:(4, 6), full:True, ov:True, box:(0, 1344, 1152, 2112)
slash_reverse:rows,cols:(4, 6), full:True, ov:True, box:(0, 2112, 1152, 2880)
thrust:rows,cols:(4, 8), full:True, ov:True, box:(0, 2880, 1536, 3648)


In [18]:
crystal_staff_crop_order = [
    (Action.spellcast,0,0),
    (Action.thrust,0,0), 
    (Action.walk,1,0),
    (Action.slash,0,0), 
    (Action.shoot,0,0), 
    (Action.hurt,1,0),
    (Action.thrust,1,1)]
split_png('crystal_staff','WEAPON', crystal_staff_crop_order)

spellcast:rows,cols:(4, 7), full:False, ov:False, box:(0, 0, 448, 256)
thrust:rows,cols:(4, 8), full:False, ov:False, box:(0, 256, 512, 512)
walk:rows,cols:(4, 9), full:True, ov:False, box:(0, 512, 576, 768)
slash:rows,cols:(4, 6), full:False, ov:False, box:(0, 768, 384, 1024)
shoot:rows,cols:(4, 13), full:False, ov:False, box:(0, 1024, 832, 1280)
hurt:rows,cols:(1, 6), full:True, ov:False, box:(0, 1280, 384, 1344)
thrust:rows,cols:(4, 8), full:True, ov:True, box:(0, 1344, 1536, 2112)


In [ ]:
#debug split_png
def test_split_png(src_file,crop_box,dest_file): #ex. 'long_sword.png',(0, 1344, 1152, 2112),'long_sword_slash_oversize.png'
    src_path = Path('.') / 'textures' / 'png' / '_source' / src_file
    test_img = Image.open(src_path.as_posix())
    cropped_img = test_img.crop(crop_box)
    save_path = Path('.') / 'textures' / 'png' / '_source' / dest_file
    cropped_img.save(save_path.as_posix())

In [20]:
import arcade
foo = arcade.AnimationKeyframe(1,5,None)
foo.duration

5